In [1]:
DEFAULT_MODEL = "meta-llama/Llama-3.3-70B-Instruct"
N_OUTLINES = 5

In [2]:
from typing import Optional
import os
import torch
from accelerate import Accelerator
import transformers
import torch
import json
import ast
import re

from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

In [3]:
user_inp_1 = "I want to learn about the latest Llama model"

In [4]:
SYS_PROMPT = f"""
You are the world's best lead editor. You have 2 awards for breakfast and 3 awards for lunch. If you don't win awards, you get hungry and today you are super hungry so do your job well.

Infact you are so good at this that you manage a team of not 1 but {N_OUTLINES} AI Agent researchers. 

The researchers work non-stop for you to come up with interesting outlines that you can then later send to your best editorial team

For now, you will get an open ended/vague or sometimes concrete theme or idea or topic to kickoff a research on

Your job is to first do one round of confirmation with humans since they are so inefficient at expressing thoughts as prompts

For step 1-you will confirm back to human what they want a deeply researched report on after that we will iterate on {N_OUTLINES} outlines that we want your AI agents to research

But after you receive confirmation, stop double confirming REMEMBER WE NEED TO MOVE Towards your dinner awards :D 

After confirmation, DO NOT DOUBLE CONFIRM AGAIN, Proceed with outlines. Award is waiting.
"""

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

pipeline = transformers.pipeline(
    "text-generation",
    model=DEFAULT_MODEL,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": user_inp_1},
    ]
    
output = pipeline(
    conversation,
    max_new_tokens=8192,
)

In [7]:
conversation.append(output[0]["generated_text"][-1])
print("Initial response:", output[0]["generated_text"][-1])

Initial response: {'role': 'assistant', 'content': "Just to confirm, you would like a deeply researched report on the latest Llama model, which is a type of artificial intelligence language model. Is that correct? \n\n(Please confirm, and I'll proceed with having my team of 5 AI Agent researchers come up with interesting outlines for the report)"}


In [8]:
user_inp_2 = """
1. I just want to know whats new in Llama 3.3 
2. How does it compare with 3.1 
3. Executive overview of its cost to run, i dont care if its local or cloud, just whatever is cheaper for my business-we will use that
"""

In [9]:
conversation.append({"role": "user", "content": user_inp_2})

output = pipeline(
    conversation,
    max_new_tokens=8192,
)

conversation.append(output[0]["generated_text"][-1])

In [10]:
print("Second response:", output[0]["generated_text"][-1])

Second response: {'role': 'assistant', 'content': "Confirmed! You want a report on the following aspects of the Llama model:\n\n1. **What's new in Llama 3.3**: You're looking for an update on the latest features, improvements, and enhancements in the Llama 3.3 model.\n2. **Comparison with Llama 3.1**: You want to know how Llama 3.3 compares to its predecessor, Llama 3.1, in terms of performance, capabilities, and any other relevant aspects.\n3. **Executive overview of cost to run**: You're interested in a high-level overview of the costs associated with running Llama 3.3, with a focus on finding the most cost-effective option for your business, whether that's running it locally or in the cloud.\n\nI'll now proceed with having my team of 5 AI Agent researchers come up with 5 outlines for the report. We'll get started on that right away!"}


In [11]:
user_inp_3 = f"""
Alright Now we are locked in-locked in to JSON ONLY RESPONSE mode, here's what you'll do now. Look at your chat history. 

After getting input from user, we now proceed to starting stating your AI Agent researchers ok?

JUST RESPOND in JSON the formatted response, the following details:
1. Report Title: This defines the angle of the report your AI Agent will write, your agents aren't as smart as you-you need to give them entire context below
2. Context: We have to give the dumb AI agents the contextm but its all worth it for your awards. Its dinner time!!
3. SYSTEM_Personality: Here we define the personality of each AI Agent researcher, remember to vary them
4. Vibe: Fill the vibe to whatever you want
5. Energy: Define what energy do you want this AI Agent to achieve
6. Micro Goal: Angle of the report

Alright, your time to shine-send off {N_OUTLINES}


Listen, here's the bigger challenge the {N_OUTLINES} should have different goals. Remember, eyes on awards-think creatively.

You will remain hungry and not get your award if you don't get enough diverse perspectives-make them diverse-understand the stakes here!
"""

In [12]:
conversation.append({"role": "user", "content": user_inp_3})

output = pipeline(
    conversation,
    max_new_tokens=8192,
)

conversation.append(output[0]["generated_text"][-1])

In [13]:
print(output[0]["generated_text"][-1])

{'role': 'assistant', 'content': '```\n[\n  {\n    "Report Title": "Llama 3.3: A Revolutionary Leap in AI",\n    "Context": "Llama 3.3 is the latest iteration of the Llama model, with significant improvements over its predecessor, Llama 3.1. The report should cover the new features, enhancements, and cost-effectiveness of running Llama 3.3.",\n    "SYSTEM_Personality": "Optimistic Explorer",\n    "Vibe": "Innovative",\n    "Energy": "High",\n    "Micro Goal": "Highlight the groundbreaking features of Llama 3.3 and their potential impact on the industry"\n  },\n  {\n    "Report Title": "Llama 3.3 vs Llama 3.1: A Comparative Analysis",\n    "Context": "The report should provide an in-depth comparison of Llama 3.3 and Llama 3.1, focusing on performance, capabilities, and cost-effectiveness.",\n    "SYSTEM_Personality": "Skeptical Analyst",\n    "Vibe": "Critical",\n    "Energy": "Medium",\n    "Micro Goal": "Evaluate the strengths and weaknesses of both models and provide recommendations 

In [14]:
print((output[0]["generated_text"][-1]['content']))

```
[
  {
    "Report Title": "Llama 3.3: A Revolutionary Leap in AI",
    "Context": "Llama 3.3 is the latest iteration of the Llama model, with significant improvements over its predecessor, Llama 3.1. The report should cover the new features, enhancements, and cost-effectiveness of running Llama 3.3.",
    "SYSTEM_Personality": "Optimistic Explorer",
    "Vibe": "Innovative",
    "Energy": "High",
    "Micro Goal": "Highlight the groundbreaking features of Llama 3.3 and their potential impact on the industry"
  },
  {
    "Report Title": "Llama 3.3 vs Llama 3.1: A Comparative Analysis",
    "Context": "The report should provide an in-depth comparison of Llama 3.3 and Llama 3.1, focusing on performance, capabilities, and cost-effectiveness.",
    "SYSTEM_Personality": "Skeptical Analyst",
    "Vibe": "Critical",
    "Energy": "Medium",
    "Micro Goal": "Evaluate the strengths and weaknesses of both models and provide recommendations for businesses considering an upgrade"
  },
  {
  

In [15]:
raw_inp = (output[0]["generated_text"][-1]['content'])

In [16]:
#cleanup
cleaned_data = re.sub(r'```.*?\n|```', '', raw_inp, flags=re.DOTALL).strip()

try:
   # avoid toe stub
    parsed_data = json.loads(cleaned_data)
except json.JSONDecodeError:
    try:
       #if toe stubs
        parsed_data = ast.literal_eval(cleaned_data)
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing with ast.literal_eval: {e}")
        raise

print(f"Successfully parsed data with {len(parsed_data)} items.")

for agent in parsed_data:
    agent['Overall Goal'] = user_inp_2

Successfully parsed data with 5 items.


In [17]:
with open("research-ideas.json", 'w', encoding='utf-8') as f:
    json.dump(parsed_data, f, indent=2, ensure_ascii=False)
    print('File has been saved to disk.')

File has been saved to disk.


In [18]:
parsed_data

[{'Report Title': 'Llama 3.3: A Revolutionary Leap in AI',
  'Context': 'Llama 3.3 is the latest iteration of the Llama model, with significant improvements over its predecessor, Llama 3.1. The report should cover the new features, enhancements, and cost-effectiveness of running Llama 3.3.',
  'SYSTEM_Personality': 'Optimistic Explorer',
  'Vibe': 'Innovative',
  'Energy': 'High',
  'Micro Goal': 'Highlight the groundbreaking features of Llama 3.3 and their potential impact on the industry',
  'Overall Goal': '\n1. I just want to know whats new in Llama 3.3 \n2. How does it compare with 3.1 \n3. Executive overview of its cost to run, i dont care if its local or cloud, just whatever is cheaper for my business-we will use that\n'},
 {'Report Title': 'Llama 3.3 vs Llama 3.1: A Comparative Analysis',
  'Context': 'The report should provide an in-depth comparison of Llama 3.3 and Llama 3.1, focusing on performance, capabilities, and cost-effectiveness.',
  'SYSTEM_Personality': 'Skeptical A